In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import seaborn as sns
nltk.download('rslp')
from nltk.stem import RSLPStemmer
import heapq as hp
from collections import Counter,OrderedDict
import math

[nltk_data] Downloading package rslp to /home/vinicius/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
data = pd.read_csv('./results.csv')

### 1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [6]:
#generate tokens from a document
def parse(doc):
    words = ''.join(str(v) for v in doc).lower()
    return RegexpTokenizer(r'[A-zÀ-ú\d]{4,}').tokenize(words)

# produces: term:[(doc, frequency)]
#build an index for a collection of documents
def build_index_with_frequency(docs):
    index = {}
    n = 0
    for text in docs.text:
        n +=1
        tokens = parse(text)
        for token in tokens:
            if (token in index):
                hasDoc = False
                for i in range(len(index[token])):
                    tup = index[token][i]
                    if (tup[0] == n):
                        index[token][i] = (n, tup[1] + 1)
                        hasDoc = True
                if (not hasDoc):
                    index[token].append((n,1))
            else:
                index[token] = [(n,1)]
    return index
index = build_index_with_frequency(data)

### 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [11]:
# produces: term:[(doc, frequency), idf]
#build an index for a collection of documents
def build_index_with_frequency_and_idf(docs):
    index = {}
    n = 0
    M = len(docs)
    for text in docs.text:
        n +=1
        tokens = parse(text)
        for token in tokens:
            if (token in index):
                hasDoc = False
                for i in range(len(index[token])):
                    tup = index[token][i]
                    if (tup[0] == n):
                        index[token][i] = (n, tup[1] + 1)
                        hasDoc = True
                if (not hasDoc):
                    index[token].append((n,1))
            else:
                index[token] = [(n,1)]
    
    for elem in index:
        k = len(index[elem])
        idf = math.log((M+1)/k)
        index[elem].append(idf)
    return index

index = build_index_with_frequency_and_idf(data)

### 3. Implementar as seguintes versões do modelo vetorial:

#### Representação binária:

In [26]:
def binary_rep(query, doc):
    score = 0
    query_t = query.split()
    doc_t = doc.split()
    for t in query_t:
        score += (t in doc_t)
    return score
#for el in data.text:
#    print(binary_vsm("deputado federal", el))

#### TF

In [31]:
def tf(query, doc):
    score = 0
    doc_t = doc.split()
    query_t = query.split()
    for t in query_t:
        score += doc_t.count(t)
    return score
#for el in data.text:
#    print(tf("deputado federal", el))

#### TF-IDF

In [35]:
def tfidf(query, doc, index):
    score = 0
    doc_t = doc.split()
    query_t = query.split()
    for t in query_t:
        count = doc_t.count(t)
        if t in index:
            score += count * index[t][-1]
    return round(score, 2)
#index = build_index_with_frequency_and_idf(data)
#for el in data.text:
#    print(tfidf("deputado federal", el, index))

#### BM25

In [40]:
def bm25(query, doc, index):
    k = 1
    M = len(data)
    doc_t = doc.split()
    query_t = query.split()
    words = list(set(doc_t) & set(query_t))
    score = 0
    for word in words:
        cwq = query_t.count(word)
        cwd = doc_t.count(word)
        dfw = len(index[word][:-1])
        y = ((k+1)*cwd)/(cwd+k)
        score += cwq * y * math.log((M+1)/dfw)
    return score
#index = build_index_with_frequency_and_idf(data)
#for el in data.text:
#    print(bm25("deputado federal", el, index))

### 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta

In [94]:
queries = ['campeonato brasileiro', 'ministério público', 'jair bolsonaro']
def top_5(query, index):
    doc_id = 0
    bin_res = []
    tf_res = []
    tdidf_res = []
    bm25_res = []
    for doc in data.text:
        bin_res.append((binary_rep(query, doc), doc_id))
        tf_res.append((tf(query, doc), doc_id))
        tdidf_res.append((tfidf(query, doc, index), doc_id))
        bm25_res.append((bm25(query, doc, index), doc_id))
        doc_id += 1
    bin_res = sorted(bin_res, reverse=True)
    tf_res = sorted(tf_res, reverse=True)
    tdidf_res = sorted(tdidf_res, reverse=True)
    bm25_res = sorted(bm25_res, reverse=True)
    return bin_res[:5], tf_res[:5], tdidf_res[:5], bm25_res[:5]
top5_binary = [0,0,0]
top5_tf = [0,0,0]
top5_tfidf = [0,0,0]
top5_bm25 = [0,0,0]
for i in range(len(queries)):
    q = queries[i]
    top5_binary[i], top5_tf[i], top5_tfidf[i], top5_bm25[i] = top_5(q, index)

pd.options.display.max_colwidth = 500  
df = pd.DataFrame()

df['Query'] = queries
df['Binary'] = top5_binary
df['TF'] = top5_tf
df['TF-IDF'] = top5_tfidf
df['BM25'] = top5_bm25

display(df)

,Query,Binary,TF,TF-IDF,BM25
0,campeonato brasileiro,"[(2, 54), (1, 237), (1, 224), (1, 223), (1, 222)]","[(6, 54), (4, 163), (4, 104), (4, 41), (3, 209)]","[(15.73, 54), (6.06, 163), (6.06, 104), (6.06, 41), (4.54, 209)]","[(7.86574377189595, 54), (3.7297014486341915, 55), (3.7297014486341915, 46), (3.7297014486341915, 44), (2.422604372207641, 163)]"
1,ministério público,"[(2, 216), (2, 208), (1, 246), (1, 245), (1, 242)]","[(10, 196), (4, 216), (4, 198), (4, 109), (4, 55)]","[(12.45, 196), (5.49, 216), (5.28, 123), (4.98, 198), (4.98, 109)]","[(3.6274530004379706, 216), (3.005055601014875, 208), (2.6403912032530257, 123), (2.3470144028915785, 220), (2.3470144028915785, 114)]"
2,jair bolsonaro,"[(0, 248), (0, 247), (0, 246), (0, 245), (0, 244)]","[(0, 248), (0, 247), (0, 246), (0, 245), (0, 244)]","[(0.0, 248), (0.0, 247), (0.0, 246), (0.0, 245), (0.0, 244)]","[(0, 248), (0, 247), (0, 246), (0, 245), (0, 244)]"


### 5. Compare os resultados encontrados e responda

#### 1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta

In [62]:
pd.options.display.max_colwidth = 500
def print_docs(docs):
    results_title = []
    results_subtitle = []
    results_url = []
    for doc in docs:
        doc_index = doc-1
        results_title.append(data.title[doc_index])
        results_subtitle.append(data.subtitle[doc_index])
        results_url.append(data.url[doc_index])
    df_query_0 = pd.DataFrame()
    df_query_0["title"] = list(results_title)
    df_query_0["subtitle"] = list(results_subtitle)
    df_query_0["url"] = list(results_url)
    display(df_query_0)
    print("\n")

##### query1: campeonato brasileiro

In [81]:
binary_docs1 = []
tf_docs1 = []
tfidf_docs1 = []
bm25_docs1 = []

for i in range(5):
    binary_docs1.append(top5_binary[0][i][1])
    tf_docs1.append(top5_tf[0][i][1])
    tfidf_docs1.append(top5_tfidf[0][i][1])
    bm25_docs1.append(top5_bm25[0][i][1])

###### binary

In [82]:
print_docs(binary_docs1)

,title,subtitle,url
0,Sorteio coloca Manchester United contra o Barcelona nas quartas da Champions League,Ajax x Juventus Liverpool x Porto e Tottenham x Manchester City são os outros confrontos. Sorteio também definiu chaveamento até a final,https://brasil.elpais.com/brasil/2019/03/14/deportes/1552585179_394268.html
1,Bolsonaro anuncia 13º salário para o Bolsa Família para marcar 100 dias no poder,Presidente fez anúncio em 'live' no Facebook nesta quinta-feira ao lado de Sergio Moro,https://brasil.elpais.com/brasil/2019/04/05/politica/1554416098_449063.html
2,A regra de sangue da Operação Condor a aliança mortífera das ditaduras do Cone Sul,A Operação Condor aliança entre Brasil Argentina Chile Uruguai Paraguai e Bolívia permitiu a troca de informações e livre trânsito para perseguir torturar e matar opositores da ditadura no continente,https://brasil.elpais.com/brasil/2019/03/29/politica/1553895462_193096.html
3,“Não me arrependo de nada”,Procurador do caso Herzog que acobertou torturadores e pediu pena de morte para militantes Durval Ayrton Moura de Araújo de 99 anos conversa com o EL PAÍS,https://brasil.elpais.com/brasil/2019/03/28/politica/1553789942_315053.html
4,A jovem deputada que jogou contra as cordas o ministro da Educação e sua “lista de desejos”,Neófita no Congresso Tabata Amaral (PDT-SP) sugere que Ricardo Vélez se demita do MEC o deixa em silêncio e agrava ainda mais a situação do ministro olavista de Bolsonaro,https://brasil.elpais.com/brasil/2019/03/28/politica/1553801628_869241.html


###### TF

In [83]:
print_docs(tf_docs1)

,title,subtitle,url
0,Sorteio coloca Manchester United contra o Barcelona nas quartas da Champions League,Ajax x Juventus Liverpool x Porto e Tottenham x Manchester City são os outros confrontos. Sorteio também definiu chaveamento até a final,https://brasil.elpais.com/brasil/2019/03/14/deportes/1552585179_394268.html
1,A era dos reptilianos,As teorias da conspiração pressupõem que os dois lados de uma disputa científica ou social devem ter a mesma veracidade,https://brasil.elpais.com/brasil/2019/03/21/opinion/1553190541_750395.html
2,Cúpula militar uruguaia cai após revelação de crimes da ditadura,Presidente reage a um escândalo relacionado a crimes de lesa-humanidade cometidos durante a ditadura,https://brasil.elpais.com/brasil/2019/04/01/internacional/1554141574_437551.html
3,Turim se apaixona por Cristiano Ronaldo,Juventus o blindou diante do caso de suposto estupro e ele oferece ao time sua melhor versão.,https://brasil.elpais.com/brasil/2019/02/17/deportes/1550427113_361941.html
4,Paulo Guedes cancela ida à Câmara para defender reforma da Previdência e acirra tensão com o Legislativo,Ministro da Economia ia em 'missão de paz' para explicar projeto de reforma na CCJ e tentar atenuar crise entre Governo e Rodrigo Maia. Desistência de Guedes foi criticada,https://brasil.elpais.com/brasil/2019/03/26/politica/1553607037_149988.html


###### TF-IDF

In [84]:
print_docs(tfidf_docs1)

,title,subtitle,url
0,Sorteio coloca Manchester United contra o Barcelona nas quartas da Champions League,Ajax x Juventus Liverpool x Porto e Tottenham x Manchester City são os outros confrontos. Sorteio também definiu chaveamento até a final,https://brasil.elpais.com/brasil/2019/03/14/deportes/1552585179_394268.html
1,A era dos reptilianos,As teorias da conspiração pressupõem que os dois lados de uma disputa científica ou social devem ter a mesma veracidade,https://brasil.elpais.com/brasil/2019/03/21/opinion/1553190541_750395.html
2,Cúpula militar uruguaia cai após revelação de crimes da ditadura,Presidente reage a um escândalo relacionado a crimes de lesa-humanidade cometidos durante a ditadura,https://brasil.elpais.com/brasil/2019/04/01/internacional/1554141574_437551.html
3,Turim se apaixona por Cristiano Ronaldo,Juventus o blindou diante do caso de suposto estupro e ele oferece ao time sua melhor versão.,https://brasil.elpais.com/brasil/2019/02/17/deportes/1550427113_361941.html
4,Paulo Guedes cancela ida à Câmara para defender reforma da Previdência e acirra tensão com o Legislativo,Ministro da Economia ia em 'missão de paz' para explicar projeto de reforma na CCJ e tentar atenuar crise entre Governo e Rodrigo Maia. Desistência de Guedes foi criticada,https://brasil.elpais.com/brasil/2019/03/26/politica/1553607037_149988.html


###### BM25

In [85]:
print_docs(bm25_docs1)

,title,subtitle,url
0,Sorteio coloca Manchester United contra o Barcelona nas quartas da Champions League,Ajax x Juventus Liverpool x Porto e Tottenham x Manchester City são os outros confrontos. Sorteio também definiu chaveamento até a final,https://brasil.elpais.com/brasil/2019/03/14/deportes/1552585179_394268.html
1,Eder Militão o zagueiro mais caro da história do Real Madrid e primeiro reforço de Zidane,Revelado pelo São Paulo o versátil defensor foi comprado do Porto por 50 milhões de euros e chega ao fim da temporada,https://brasil.elpais.com/brasil/2019/03/14/deportes/1552561571_389016.html
2,O trágico destino de Tom Ballard,Filho da alpinista Alison Hargreaves morta no pico do K2 em 1995 o inglês perdeu o contato ao tentar uma ascensão invernal do Nanga Parbat (8.125 metros) com o italiano Daniele Nardi,https://brasil.elpais.com/brasil/2019/03/02/deportes/1551530086_689843.html
3,Onde e como assistir a Real Madrid x Barcelona pelo Campeonato Espanhol,Depois da vitória catalã na Copa rivais voltam a se encontrar no Santiago Bernabéu pela Liga,https://brasil.elpais.com/brasil/2019/03/01/deportes/1551476564_723854.html
4,A era dos reptilianos,As teorias da conspiração pressupõem que os dois lados de uma disputa científica ou social devem ter a mesma veracidade,https://brasil.elpais.com/brasil/2019/03/21/opinion/1553190541_750395.html


Observando os títulos, podemos ver que o modelo **BM25** se saiu melhor nessa query, pois possui mais resultados de notícias relacionadas ao esporte do que os outros modelos.

##### query2: ministério público

In [87]:
binary_docs2 = []
tf_docs2 = []
tfidf_docs2 = []
bm25_docs2 = []

for i in range(5):
    binary_docs2.append(top5_binary[1][i][1])
    tf_docs2.append(top5_tf[1][i][1])
    tfidf_docs2.append(top5_tfidf[1][i][1])
    bm25_docs2.append(top5_bm25[1][i][1])

###### binary

In [88]:
print_docs(binary_docs2)

,title,subtitle,url
0,Um Brasil de dedo em riste às portas do Mackenzie,A expectativa de uma visita do presidente Jair Bolsonaro à universidade em São Paulo cenário emblemático da luta estudantil durante a ditadura gerou embates inflamados entre estudantes e bolsonaristas,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html
1,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,"Com medida simbólica presidente determinou que Forças Armadas façam ""comemorações devidas"" da data",https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html
2,Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo,Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553587140_841439.html
3,Forças curdo-árabes anunciam a queda do último bastião do ISIS na Síria,As milícias da coalizão anunciaram na manhã de sábado o fim do califado do Estado Islâmico depois de quase cinco anos de combates,https://brasil.elpais.com/brasil/2019/03/23/internacional/1553329466_793785.html
4,‘Malhação’ joga luz sobre o abuso sexual no futebol,Alerta de assédio a jovens atletas em categorias de base chega à TV mas ainda é tabu dentro dos clubes,https://brasil.elpais.com/brasil/2019/03/18/deportes/1552944014_726009.html


###### TF

In [92]:
print_docs(tf_docs2)

,title,subtitle,url
0,Tudo funciona bem em Mick Jagger menos o coração,O cantor de 75 anos se cuida mais que nunca. Pratica esporte medita e não bebe mas isso não evitou que tenha de ser operado por um problema cardíaco,https://brasil.elpais.com/brasil/2019/04/02/cultura/1554221535_416406.html
1,Um Brasil de dedo em riste às portas do Mackenzie,A expectativa de uma visita do presidente Jair Bolsonaro à universidade em São Paulo cenário emblemático da luta estudantil durante a ditadura gerou embates inflamados entre estudantes e bolsonaristas,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html
2,Todos os filmes de Tim Burton do pior ao melhor,Aproveitamos a recente estreia de 'Dumbo' para pedir esse ranking a Rubén Romero crítico de cinema professor de Comunicação Audiovisual da Universidade Carlos III e especialista no cinema do diretor que gosta de jovens estranhos,https://brasil.elpais.com/brasil/2019/03/28/cultura/1553773473_512189.html
3,A ascensão imparável das ‘cholitas escaladoras’,Símbolo da luta pela visibilidade das indígenas na Bolívia um grupo de mulheres aimarás acaba de completar seu último desafio: chegar ao cume do Aconcágua o mais alto da América,https://brasil.elpais.com/brasil/2019/03/22/internacional/1553277982_341963.html
4,Eder Militão o zagueiro mais caro da história do Real Madrid e primeiro reforço de Zidane,Revelado pelo São Paulo o versátil defensor foi comprado do Porto por 50 milhões de euros e chega ao fim da temporada,https://brasil.elpais.com/brasil/2019/03/14/deportes/1552561571_389016.html


###### TF-IDF

In [90]:
print_docs(tfidf_docs2)

,title,subtitle,url
0,Tudo funciona bem em Mick Jagger menos o coração,O cantor de 75 anos se cuida mais que nunca. Pratica esporte medita e não bebe mas isso não evitou que tenha de ser operado por um problema cardíaco,https://brasil.elpais.com/brasil/2019/04/02/cultura/1554221535_416406.html
1,Um Brasil de dedo em riste às portas do Mackenzie,A expectativa de uma visita do presidente Jair Bolsonaro à universidade em São Paulo cenário emblemático da luta estudantil durante a ditadura gerou embates inflamados entre estudantes e bolsonaristas,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html
2,OIT alerta para precarização do emprego em nível mundial,Trabalho temporário volta a crescer na Europa mesmo após reformas. Na América Latina recuperação econômica terá efeito modesto na geração de emprego,https://brasil.elpais.com/brasil/2019/02/13/economia/1550088419_496910.html
3,Todos os filmes de Tim Burton do pior ao melhor,Aproveitamos a recente estreia de 'Dumbo' para pedir esse ranking a Rubén Romero crítico de cinema professor de Comunicação Audiovisual da Universidade Carlos III e especialista no cinema do diretor que gosta de jovens estranhos,https://brasil.elpais.com/brasil/2019/03/28/cultura/1553773473_512189.html
4,A ascensão imparável das ‘cholitas escaladoras’,Símbolo da luta pela visibilidade das indígenas na Bolívia um grupo de mulheres aimarás acaba de completar seu último desafio: chegar ao cume do Aconcágua o mais alto da América,https://brasil.elpais.com/brasil/2019/03/22/internacional/1553277982_341963.html


###### BM25

In [91]:
print_docs(bm25_docs2)

,title,subtitle,url
0,Um Brasil de dedo em riste às portas do Mackenzie,A expectativa de uma visita do presidente Jair Bolsonaro à universidade em São Paulo cenário emblemático da luta estudantil durante a ditadura gerou embates inflamados entre estudantes e bolsonaristas,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html
1,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,"Com medida simbólica presidente determinou que Forças Armadas façam ""comemorações devidas"" da data",https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html
2,OIT alerta para precarização do emprego em nível mundial,Trabalho temporário volta a crescer na Europa mesmo após reformas. Na América Latina recuperação econômica terá efeito modesto na geração de emprego,https://brasil.elpais.com/brasil/2019/02/13/economia/1550088419_496910.html
3,Ferrovia Norte-Sul vale três vezes mais que previsto em edital aponta estudo,Linha considerada a espinha dorsal do desenvolvimento logístico do país deve ser leiloada nesta quinta-feira com preço menor do que sugerido pela estatal Valec. Há duas empresas interessadas.,https://brasil.elpais.com/brasil/2019/03/28/politica/1553729926_061804.html
4,Vozes recuperadas da ditadura argentina,Familiares e amigos de vítimas narram em primeira pessoa as últimas horas com vida de entes queridos,https://brasil.elpais.com/brasil/2019/03/29/internacional/1553885143_703482.html


Novamente o **BM25** apresentou os melhores resultados, trazendo notícias relacionadas á política e economia em todos os seus resultados.

##### query3: jair bolsonaro

In [95]:
binary_docs3 = []
tf_docs3 = []
tfidf_docs3 = []
bm25_docs3 = []

for i in range(5):
    binary_docs3.append(top5_binary[2][i][1])
    tf_docs3.append(top5_tf[2][i][1])
    tfidf_docs3.append(top5_tfidf[2][i][1])
    bm25_docs3.append(top5_bm25[2][i][1])

###### binary

In [97]:
print_docs(binary_docs3)

,title,subtitle,url
0,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Para filósofa e matemática campo progressista deve se opor à reforma da Previdência de Bolsonaro apresentando seu próprio projeto. Defende que a renovação deve vir a partir dos municípios,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html
1,Dois aviões russos chegam à Venezuela com uma centena de militares,Aterrissagem das aeronaves ocorre depois de Maduro dizer que havia reforçado sua segurança,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html
2,Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo,Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553587140_841439.html
3,Forças curdo-árabes anunciam a queda do último bastião do ISIS na Síria,As milícias da coalizão anunciaram na manhã de sábado o fim do califado do Estado Islâmico depois de quase cinco anos de combates,https://brasil.elpais.com/brasil/2019/03/23/internacional/1553329466_793785.html
4,Nanga Parbat a obsessão com a ‘montanha assassina’,A recente morte de dois alpinistas no topo mais letal depois do Annapurna acentua a angustiante luta com este cume há mais de um século,https://brasil.elpais.com/brasil/2019/03/20/deportes/1553098084_373740.html


###### TF

In [98]:
print_docs(tf_docs3)

,title,subtitle,url
0,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Para filósofa e matemática campo progressista deve se opor à reforma da Previdência de Bolsonaro apresentando seu próprio projeto. Defende que a renovação deve vir a partir dos municípios,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html
1,Dois aviões russos chegam à Venezuela com uma centena de militares,Aterrissagem das aeronaves ocorre depois de Maduro dizer que havia reforçado sua segurança,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html
2,Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo,Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553587140_841439.html
3,Forças curdo-árabes anunciam a queda do último bastião do ISIS na Síria,As milícias da coalizão anunciaram na manhã de sábado o fim do califado do Estado Islâmico depois de quase cinco anos de combates,https://brasil.elpais.com/brasil/2019/03/23/internacional/1553329466_793785.html
4,Nanga Parbat a obsessão com a ‘montanha assassina’,A recente morte de dois alpinistas no topo mais letal depois do Annapurna acentua a angustiante luta com este cume há mais de um século,https://brasil.elpais.com/brasil/2019/03/20/deportes/1553098084_373740.html


###### TF-IDF

In [99]:
print_docs(tfidf_docs3)

,title,subtitle,url
0,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Para filósofa e matemática campo progressista deve se opor à reforma da Previdência de Bolsonaro apresentando seu próprio projeto. Defende que a renovação deve vir a partir dos municípios,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html
1,Dois aviões russos chegam à Venezuela com uma centena de militares,Aterrissagem das aeronaves ocorre depois de Maduro dizer que havia reforçado sua segurança,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html
2,Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo,Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553587140_841439.html
3,Forças curdo-árabes anunciam a queda do último bastião do ISIS na Síria,As milícias da coalizão anunciaram na manhã de sábado o fim do califado do Estado Islâmico depois de quase cinco anos de combates,https://brasil.elpais.com/brasil/2019/03/23/internacional/1553329466_793785.html
4,Nanga Parbat a obsessão com a ‘montanha assassina’,A recente morte de dois alpinistas no topo mais letal depois do Annapurna acentua a angustiante luta com este cume há mais de um século,https://brasil.elpais.com/brasil/2019/03/20/deportes/1553098084_373740.html


###### BM25

In [100]:
print_docs(bm25_docs3)

,title,subtitle,url
0,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Para filósofa e matemática campo progressista deve se opor à reforma da Previdência de Bolsonaro apresentando seu próprio projeto. Defende que a renovação deve vir a partir dos municípios,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html
1,Dois aviões russos chegam à Venezuela com uma centena de militares,Aterrissagem das aeronaves ocorre depois de Maduro dizer que havia reforçado sua segurança,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html
2,Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo,Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553587140_841439.html
3,Forças curdo-árabes anunciam a queda do último bastião do ISIS na Síria,As milícias da coalizão anunciaram na manhã de sábado o fim do califado do Estado Islâmico depois de quase cinco anos de combates,https://brasil.elpais.com/brasil/2019/03/23/internacional/1553329466_793785.html
4,Nanga Parbat a obsessão com a ‘montanha assassina’,A recente morte de dois alpinistas no topo mais letal depois do Annapurna acentua a angustiante luta com este cume há mais de um século,https://brasil.elpais.com/brasil/2019/03/20/deportes/1553098084_373740.html


Curiosamente essa todos os modelos retornaram os mesmos resultados para essa query. Dessa forma, não podemos determinar quem obteve os melhores resultados.

#### 2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo